Can we predict energy consumption per value added for different sectors in the USA and India based on historical data from 2015 to 2021? Note: China data is not available in dataset.

https://www.iea.org/data-and-statistics/data-product/energy-and-emissions-per-value-added-database

**1. Visual Inspection:**

In [ ]:
import pandas as pd

# Load Excel files
va_section_data_file = 'VASectionData.xlsx'
india_usa_isic_rev4_file = 'IndiaUSAISICrev4.xlsx'
india_usa_indicators_file = 'IndiaUSAIndicators.xlsx'
india_usa_mfg_file = 'IndiaUSAMfg.xlsx'

# Load the sheets
va_section_data_df = pd.read_excel(va_section_data_file, sheet_name=0)
isic_rev4_df = pd.read_excel(india_usa_isic_rev4_file)
indicators_df = pd.read_excel(india_usa_indicators_file)
mfg_df = pd.read_excel(india_usa_mfg_file)

# Save the sheets as CSV files
va_section_data_df.to_csv('VASectionData.csv', index=False)
isic_rev4_df.to_csv('IndiaUSAISICrev4.csv', index=False)
indicators_df.to_csv('IndiaUSAIndicators.csv', index=False)
mfg_df.to_csv('IndiaUSAMfg.csv', index=False)

# Print the index of the DataFrames (optional)
print(va_section_data_df.index)
print(isic_rev4_df.index)
print(indicators_df.index)
print(mfg_df.index)

# Print the columns of the DataFrames (optional)
print(va_section_data_df.columns)
print(isic_rev4_df.columns)
print(indicators_df.columns)
print(mfg_df.columns)


RangeIndex(start=0, stop=90494, step=1)
RangeIndex(start=0, stop=134, step=1)
RangeIndex(start=0, stop=21448, step=1)
RangeIndex(start=0, stop=528, step=1)
Index(['Country ISO3', 'Country', 'Year', 'ISIC Section', 'ISIC Division',
       'ISIC Division Desc', 'Measure', 'Method',
       'Value LCU (Million national currency)', 'Value USD (Million USD)',
       'Source'],
      dtype='object')
Index(['ISIC section', 'ISIC division', 'ISIC division description',
       'ISIC aggregation level'],
      dtype='object')
Index(['Country ISO3', 'Country', 'Year', 'ISIC Division',
       'ISIC division description', 'Indicator', 'Value', 'Unit'],
      dtype='object')
Index(['Country ISO3', 'Country', 'Year', 'ISIC Division',
       'ISIC Division Desc', 'Measure', 'Method',
       'Value LCU (million national currency)', 'Value USD (million USD)'],
      dtype='object')


In [ ]:
# Load the sheets as DataFrames
va_section_data_df = pd.read_csv('VASectionData.csv')
isic_rev4_df = pd.read_csv('IndiaUSAISICrev4.csv')
indicators_df = pd.read_csv('IndiaUSAIndicators.csv')
mfg_df = pd.read_csv('IndiaUSAMfg.csv')

# Print the column names for each DataFrame
print("VASectionData Columns:")
print(va_section_data_df.columns)

print("\nISICrev4 Columns:")
print(isic_rev4_df.columns)

print("\nIndicators Columns:")
print(indicators_df.columns)

print("\nMfg Columns:")
print(mfg_df.columns)


VASectionData Columns:
Index(['Country ISO3', 'Country', 'Year', 'ISIC Section', 'ISIC Division',
       'ISIC Division Desc', 'Measure', 'Method',
       'Value LCU (Million national currency)', 'Value USD (Million USD)',
       'Source'],
      dtype='object')

ISICrev4 Columns:
Index(['ISIC section', 'ISIC division', 'ISIC division description',
       'ISIC aggregation level'],
      dtype='object')

Indicators Columns:
Index(['Country ISO3', 'Country', 'Year', 'ISIC Division',
       'ISIC division description', 'Indicator', 'Value', 'Unit'],
      dtype='object')

Mfg Columns:
Index(['Country ISO3', 'Country', 'Year', 'ISIC Division',
       'ISIC Division Desc', 'Measure', 'Method',
       'Value LCU (million national currency)', 'Value USD (million USD)'],
      dtype='object')


Delete columns we will not use.

In [ ]:
# Delete columns from va_section_data_df
va_section_data_df = va_section_data_df.drop(['Country ISO3', 'ISIC Section', 'ISIC Division', 'Method', 'Value LCU (Million national currency)', 'Source'], axis=1)

# Delete columns from isic_rev4_df
isic_rev4_df = isic_rev4_df.drop(['ISIC section', 'ISIC division'], axis=1)

# Delete columns from indicators_df
indicators_df = indicators_df.drop(['Country ISO3', 'ISIC Division'], axis=1)

# Delete columns from mfg_df
mfg_df = mfg_df.drop(['Country ISO3', 'ISIC Division', 'Method', 'Value LCU (million national currency)'], axis=1)

# Save the modified DataFrames to CSV files
va_section_data_df.to_csv('VASectionData.csv', index=False)
isic_rev4_df.to_csv('IndiaUSAISICrev4.csv', index=False)
indicators_df.to_csv('IndiaUSAIndicators.csv', index=False)
mfg_df.to_csv('IndiaUSAMfg.csv', index=False)

# Print the column names for each DataFrame
print("VASectionData Columns:")
print(va_section_data_df.columns)

print("\nISICrev4 Columns:")
print(isic_rev4_df.columns)

print("\nIndicators Columns:")
print(indicators_df.columns)

print("\nMfg Columns:")
print(mfg_df.columns)



VASectionData Columns:
Index(['Country', 'Year', 'ISIC Division Desc', 'Measure',
       'Value USD (Million USD)'],
      dtype='object')

ISICrev4 Columns:
Index(['ISIC division description', 'ISIC aggregation level'], dtype='object')

Indicators Columns:
Index(['Country', 'Year', 'ISIC division description', 'Indicator', 'Value',
       'Unit'],
      dtype='object')

Mfg Columns:
Index(['Country', 'Year', 'ISIC Division Desc', 'Measure',
       'Value USD (million USD)'],
      dtype='object')


In [ ]:
# Print column names for VASectionData
print("VASectionData Columns:")
print(va_section_data_df.columns)

# Print column names for ISICrev4
print("\nISICrev4 Columns:")
print(isic_rev4_df.columns)

# Print column names for Indicators
print("\nIndicators Columns:")
print(indicators_df.columns)

# Print column names for Mfg
print("\nMfg Columns:")
print(mfg_df.columns)


VASectionData Columns:
Index(['Country ISO3', 'Country', 'Year', 'ISIC Section', 'ISIC Division',
       'ISIC Division Desc', 'Measure', 'Method',
       'Value LCU (Million national currency)', 'Value USD (Million USD)',
       'Source'],
      dtype='object')

ISICrev4 Columns:
Index(['ISIC section', 'ISIC division', 'ISIC division description',
       'ISIC aggregation level'],
      dtype='object')

Indicators Columns:
Index(['Country ISO3', 'Country', 'Year', 'ISIC Division',
       'ISIC division description', 'Indicator', 'Value', 'Unit'],
      dtype='object')

Mfg Columns:
Index(['Country ISO3', 'Country', 'Year', 'ISIC Division',
       'ISIC Division Desc', 'Measure', 'Method',
       'Value LCU (million national currency)', 'Value USD (million USD)'],
      dtype='object')


Filter India and United States

In [ ]:
# Filter VASectionData for USA and India
df_filtered_va_section_data = va_section_data_df[va_section_data_df['Country'].isin(['United States', 'India'])]

# Filter ISICrev4 for USA and India
df_filtered_isic_rev4 = isic_rev4_df  # No 'Country' column in ISICrev4 DataFrame

# Filter Indicators for USA and India
df_filtered_indicators = indicators_df[indicators_df['Country'].isin(['United States', 'India'])]

# Filter Mfg for USA and India
df_filtered_mfg = mfg_df[mfg_df['Country'].isin(['United States', 'India'])]


In [ ]:
# Print column names and first few rows for VASectionData
print("VASectionData Columns:")
print(df_filtered_va_section_data.columns)
print("\nFirst few rows:")
print(df_filtered_va_section_data.head())

# Print column names and first few rows for ISICrev4
print("\nISICrev4 Columns:")
print(df_filtered_isic_rev4.columns)
print("\nFirst few rows:")
print(df_filtered_isic_rev4.head())

# Print column names and first few rows for Indicators
print("\nIndicators Columns:")
print(df_filtered_indicators.columns)
print("\nFirst few rows:")
print(df_filtered_indicators.head())

# Print column names and first few rows for Mfg
print("\nMfg Columns:")
print(df_filtered_mfg.columns)
print("\nFirst few rows:")
print(df_filtered_mfg.head())


VASectionData Columns:
Index(['Country', 'Year', 'ISIC Division Desc', 'Measure',
       'Value USD (Million USD)'],
      dtype='object')

First few rows:
      Country  Year                                 ISIC Division Desc  \
36297   India  2000                  Agriculture, forestry and fishing   
36298   India  2000                               Mining and quarrying   
36299   India  2000                                      Manufacturing   
36300   India  2000  Electricity, gas, steam and air conditioning s...   
36301   India  2000  Water supply; sewerage, waste management and r...   

              Measure  Value USD (Million USD)  
36297  Current prices                  99132.8  
36298  Current prices                  11394.6  
36299  Current prices                  89764.6  
36300  Current prices                  12415.0  
36301  Current prices                   1692.2  

ISICrev4 Columns:
Index(['ISIC division description', 'ISIC aggregation level'], dtype='object')

First 

Explanation of Each Column

    Year: The year of the data point.
    Measure: Current Prices: Nominal values without adjustment for inflation and Constant 2015 Prices: Values adjusted for inflation to maintain the purchasing power of the currency.
    Value USD (Million USD): The value of the measure in million USD.
    Source: Organization for Economic Cooperation and Development National Accounts database.

In [ ]:
print("VASectionData Columns:")
print(va_section_data_df.columns)

print("\nIndicators Columns:")
print(indicators_df.columns)

print("\nMfg Columns:")
print(mfg_df.columns)


VASectionData Columns:
Index(['Country', 'Year', 'ISIC Division Desc', 'Measure',
       'Value USD (Million USD)'],
      dtype='object')

Indicators Columns:
Index(['Country', 'Year', 'ISIC division description', 'Indicator', 'Value',
       'Unit'],
      dtype='object')

Mfg Columns:
Index(['Country', 'Year', 'ISIC Division Desc', 'Measure',
       'Value USD (million USD)'],
      dtype='object')


In [ ]:
# Filter for the years 2015 to 2021 and 'Constant 2015 prices' in the 'Measure' column

# Filter VASectionData for 'United States' and 'India' and years 2015 to 2021 and 'Constant 2015 prices' in the 'Measure' column
df_filtered_va_section_data = va_section_data_df[(va_section_data_df['Country'].isin(['United States', 'India'])) &
                                                 (va_section_data_df['Year'].between(2015, 2021)) &
                                                 (va_section_data_df['Measure'] == 'Constant 2015 prices')]

# Filter Indicators for 'United States' and 'India' and years 2015 to 2021
df_filtered_indicators = indicators_df[(indicators_df['Country'].isin(['United States', 'India'])) &
                                       (indicators_df['Year'].between(2015, 2021))]

# Filter Mfg for 'United States' and 'India' and years 2015 to 2021 and 'Constant 2015 prices' in the 'Measure' column
df_filtered_mfg = mfg_df[(mfg_df['Country'].isin(['United States', 'India'])) &
                         (mfg_df['Year'].between(2015, 2021)) &
                         (mfg_df['Measure'] == 'Constant 2015 prices')]

# Print the first few rows of each filtered DataFrame
print("VASectionData:")
print(df_filtered_va_section_data.head())

print("\nIndicators:")
print(df_filtered_indicators.head())

print("\nMfg:")
print(df_filtered_mfg.head())


VASectionData:
      Country  Year                                 ISIC Division Desc  \
36872   India  2015                  Agriculture, forestry and fishing   
36873   India  2015                               Mining and quarrying   
36874   India  2015                                      Manufacturing   
36875   India  2015  Electricity, gas, steam and air conditioning s...   
36876   India  2015  Water supply; sewerage, waste management and r...   

                    Measure  Value USD (Million USD)  
36872  Constant 2015 prices                355740.20  
36873  Constant 2015 prices                 46495.71  
36874  Constant 2015 prices                350505.28  
36875  Constant 2015 prices                 44015.19  
36876  Constant 2015 prices                  8214.00  

Indicators:
     Country  Year          ISIC division description  \
9003   India  2015  Agriculture, forestry and fishing   
9004   India  2015               Mining and quarrying   
9005   India  2015        

In [ ]:
print("VASectionData Columns:")
print(va_section_data_df.columns)

print("\nIndiaUSAISICrev4 Columns:")
print(isic_rev4_df.columns)

print("\nIndiaUSAIndicators Columns:")
print(indicators_df.columns)

print("\nIndiaUSAMfg Columns:")
print(mfg_df.columns)


VASectionData Columns:
Index(['Country', 'Year', 'ISIC Division Desc', 'Measure',
       'Value USD (Million USD)'],
      dtype='object')

IndiaUSAISICrev4 Columns:
Index(['ISIC division description', 'ISIC aggregation level'], dtype='object')

IndiaUSAIndicators Columns:
Index(['Country', 'Year', 'ISIC division description', 'Indicator', 'Value',
       'Unit'],
      dtype='object')

IndiaUSAMfg Columns:
Index(['Country', 'Year', 'ISIC Division Desc', 'Measure',
       'Value USD (million USD)'],
      dtype='object')


**2. Data Cleaning:**

In [ ]:
# Load the filtered CSV file for analysis
# Check for null values in VASectionData
print("Null values in VASectionData:")
print(va_section_data_df.isnull().sum())

# Check for null values in IndiaUSAISICrev4
print("\nNull values in IndiaUSAISICrev4:")
print(isic_rev4_df.isnull().sum())

# Check for null values in IndiaUSAIndicators
print("\nNull values in IndiaUSAIndicators:")
print(indicators_df.isnull().sum())

# Check for null values in IndiaUSAMfg
print("\nNull values in IndiaUSAMfg:")
print(mfg_df.isnull().sum())


Null values in VASectionData:
Country                       0
Year                          0
ISIC Division Desc            0
Measure                       0
Value USD (Million USD)    3326
dtype: int64

Null values in IndiaUSAISICrev4:
ISIC division description    0
ISIC aggregation level       0
dtype: int64

Null values in IndiaUSAIndicators:
Country                      0
Year                         0
ISIC division description    0
Indicator                    0
Value                        0
Unit                         0
dtype: int64

Null values in IndiaUSAMfg:
Country                    0
Year                       0
ISIC Division Desc         0
Measure                    0
Value USD (million USD)    0
dtype: int64


In [ ]:
# Handle missing values by imputing with median
# Mean imputation for VASectionData
mean_va_section_data = df_filtered_va_section_data['Value USD (Million USD)'].mean()
df_filtered_va_section_data = df_filtered_va_section_data.copy()  # Create a copy of the DataFrame
df_filtered_va_section_data['Value USD (Million USD)'].fillna(mean_va_section_data, inplace=True)

# Check if all missing values are filled
print("Null values after mean imputation in VASectionData:")
print(df_filtered_va_section_data.isnull().sum())


Null values after mean imputation in VASectionData:
Country                    0
Year                       0
ISIC Division Desc         0
Measure                    0
Value USD (Million USD)    0
dtype: int64


Check for outliers.

In [ ]:
import matplotlib.pyplot as plt

# Function to detect outliers using IQR method
def detect_outliers_iqr(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = (data < lower_bound) | (data > upper_bound)
    return outliers

# Check for outliers in VASectionData
outliers_va_section_data = detect_outliers_iqr(df_filtered_va_section_data['Value USD (Million USD)'])
print("Outliers in VASectionData:")
print(outliers_va_section_data.sum())

# Filter out non-numeric values from 'ISIC aggregation level' column
numeric_isic_agg_level = pd.to_numeric(df_filtered_isic_rev4['ISIC aggregation level'], errors='coerce')
filtered_isic_rev4 = df_filtered_isic_rev4[~numeric_isic_agg_level.isna()]

# Check for outliers in ISICrev4
outliers_isic_rev4 = detect_outliers_iqr(filtered_isic_rev4['ISIC aggregation level'])
print("\nOutliers in ISICrev4:")
print(outliers_isic_rev4.sum())




# Check for outliers in Indicators
outliers_indicators = detect_outliers_iqr(df_filtered_indicators['Value'])
print("\nOutliers in Indicators:")
print(outliers_indicators.sum())

# Check for outliers in Mfg
outliers_mfg = detect_outliers_iqr(df_filtered_mfg['Value USD (million USD)'])
print("\nOutliers in Mfg:")
print(outliers_mfg.sum())


Outliers in VASectionData:
0

Outliers in ISICrev4:
0

Outliers in Indicators:
27

Outliers in Mfg:
7


In [ ]:
# Print outliers in Indicators
print("Outliers in Indicators:")
print(df_filtered_indicators[outliers_indicators])

# Print outliers in Mfg
print("\nOutliers in Mfg:")
print(df_filtered_mfg[outliers_mfg])


Outliers in Indicators:
             Country  Year                          ISIC division description  \
21270  United States  2015         Manufacture of paper products and printing   
21271  United States  2015  Manufacture of coke and refined petroleum prod...   
21274  United States  2015                        Manufacture of basic metals   
21286  United States  2016         Manufacture of paper products and printing   
21287  United States  2016  Manufacture of coke and refined petroleum prod...   
21289  United States  2016  Manufacture of other non-metallic mineral prod...   
21290  United States  2016                        Manufacture of basic metals   
21302  United States  2017         Manufacture of paper products and printing   
21303  United States  2017  Manufacture of coke and refined petroleum prod...   
21306  United States  2017                        Manufacture of basic metals   
21318  United States  2018         Manufacture of paper products and printing   
2131

Use Interquartile Range method to handle outliers.

In [ ]:
# Calculate the IQR for the 'Value' column
Q1 = df_filtered_indicators['Value'].quantile(0.25)
Q3 = df_filtered_indicators['Value'].quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the dataset to remove outliers
filtered_indicators_no_outliers = df_filtered_indicators[
    (df_filtered_indicators['Value'] >= lower_bound) &
    (df_filtered_indicators['Value'] <= upper_bound)
]

# Print the shape of the original and filtered datasets
print("Original Dataset Shape:", df_filtered_indicators.shape)
print("Filtered Dataset Shape:", filtered_indicators_no_outliers.shape)


Original Dataset Shape: (238, 6)
Filtered Dataset Shape: (211, 6)


Removed datasize set from 238 rows to 211 rows indicating outliers were successfully identified and removed from the dataset. We can now focus on a more representative subset of data for an analysis.

In [ ]:
# Check unique values in all four files
unique_values_va_section_data = va_section_data_df.apply(lambda x: x.unique())
print("Unique values in VASectionData:")
print(unique_values_va_section_data)

unique_values_isic_rev4 = isic_rev4_df.apply(lambda x: x.unique())
print("\nUnique values in IndiaUSAISICrev4:")
print(unique_values_isic_rev4)

unique_values_indicators = indicators_df.apply(lambda x: x.unique())
print("\nUnique values in IndiaUSAIndicators:")
print(unique_values_indicators)

unique_values_mfg = mfg_df.apply(lambda x: x.unique())
print("\nUnique values in IndiaUSAMfg:")
print(unique_values_mfg)


Unique values in VASectionData:
Country                    [ASEAN, Afghanistan, Albania, Andorra, Argenti...
Year                       [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...
ISIC Division Desc         [Agriculture, forestry and fishing, Mining and...
Measure                               [Current prices, Constant 2015 prices]
Value USD (Million USD)    [61859.0, 38303.1, 154807.2, 12307.0, 2319.7, ...
dtype: object

Unique values in IndiaUSAISICrev4:
ISIC division description    [Total, Agriculture, forestry and fishing, Agr...
ISIC aggregation level                   [Total, Section, Aggregate, Division]
dtype: object

Unique values in IndiaUSAIndicators:
Country                      [Albania, Australia, Austria, Belgium, Benin, ...
Year                         [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...
ISIC division description    [Agriculture, forestry and fishing, Mining and...
Indicator                    [Per value added carbon intensity, Per value a...
Value  

**3. Exploratory Data Analysis:**

In [ ]:
# For df_filtered_va_section_data
print("Statistical Summary for df_filtered_va_section_data:")
print(df_filtered_va_section_data.describe())

# For filtered_isic_rev4
print("\nStatistical Summary for filtered_isic_rev4:")
print(filtered_isic_rev4.describe())

# For df_filtered_indicators
print("\nStatistical Summary for df_filtered_indicators:")
print(df_filtered_indicators.describe())

# For df_filtered_mfg
print("\nStatistical Summary for df_filtered_mfg:")
print(df_filtered_mfg.describe())


Statistical Summary for df_filtered_va_section_data:
              Year  Value USD (Million USD)
count   242.000000             2.420000e+02
mean   2017.607438             6.017343e+05
std       1.862166             6.788782e+05
min    2015.000000             3.594210e+03
25%    2016.000000             8.288114e+04
50%    2018.000000             2.600123e+05
75%    2019.000000             1.001469e+06
max    2021.000000             2.359199e+06

Statistical Summary for filtered_isic_rev4:
       ISIC division description ISIC aggregation level
count                          0                      0
unique                         0                      0
top                          NaN                    NaN
freq                         NaN                    NaN

Statistical Summary for df_filtered_indicators:
              Year       Value
count   238.000000  238.000000
mean   2017.672269    3.617857
std       1.856039    5.582994
min    2015.000000    0.000000
25%    2016.000000    

    df_filtered_va_section_data:
        Year: The data spans from 2015 to 2021, with a mean year of 2017.6.
        Value USD (Million USD): The values range from $3.6 million to $2.36 billion USD, with a mean of approximately $601.7 million USD.

    filtered_isic_rev4:
        There is no data available for this dataset, as both columns ("ISIC division description" and "ISIC aggregation level") have no valid entries.

    df_filtered_indicators:
        Year: Similar to the first dataset, the data spans from 2015 to 2021, with a mean year of 2017.7.
        Value: The values range from 0 to 22.33, with a mean of approximately 3.62. This could represent some kind of indicator value, possibly related to environmental or economic metrics.

    df_filtered_mfg:
        Year: The data spans from 2015 to 2021, with a mean year of 2018.
        Value USD (million USD): The values range from $22.46 million to $747.99 million USD, with a mean of approximately $183.77 million USD. This dataset likely pertains to manufacturing-related financial figures.

Overall, these summaries give you a snapshot of the range, central tendency, and variability of the filtered data in each dataset, which can help you understand the distribution and characteristics of the data.
